In [ ]:
from paramiko import SSHClient, AutoAddPolicy
import re
import csv
import json
import time
import os 
import datetime


class Tools(object):
    
    def __init__(self, file):
            self.file = file
    
    def format_result(self, dirty_format, remove=None):
        # Realiza as substituições necessárias em todos os resultados
        dirty_format = {k: v.replace(b"\n", b",").replace(b"b'", b"'").replace(b"'", b"").decode() for k, v in dirty_format.items()}

        # Formata cada resultado conforme necessário
        formatted_results = {}
        for command in dirty_format:
            result = dirty_format[command].split(",")
            if remove and isinstance(remove, dict):
                for key in remove:
                    result = [r.replace(key, remove[key]) for r in result if r.strip()]
            formatted_results[command] = result
        return formatted_results

    def manipulation_csv_to_json(self, file, cols_names, cols_del):
        data = []
        with open(file, encoding='utf-8') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for row in csv_reader:
                row_dict = dict(zip(cols_names, row))
                # Remove as colunas especificadas em col_names
                for col_name in cols_del:
                    row_dict.pop(col_name, None)
                data.append(row_dict)
        json_data = json.dumps(data, ensure_ascii=False, indent=4)
        return json_data

    
class Manager_Linux(Tools):


    def __init__(self, gateway, port, username, password):
        self.gateway = gateway
        self.username = username
        self.password = password
        self.port = port

    def connection_linux(self):
        self.ssh = SSHClient()
        self.ssh.load_system_host_keys()
        self.ssh.set_missing_host_key_policy(AutoAddPolicy())
        return self.ssh.connect(self.gateway, port=self.port, username=self.username, password=self.password, timeout=2)
    
    def connection_sftp(self):
        self.connection_linux()
        self.sftp_client = self.ssh.open_sftp()
        return self.sftp_client

    def command_gateway(self, command):
        try:
            self.connection_linux()
            stdin, stdout, stderr = self.ssh.exec_command(command)
            return stdout.read()
        except TimeoutError as timeout:
            return TimeoutError
        
    def backup_gateway(self, name_gateway):
        self.date = datetime.datetime.now()
        self.connection_sftp()
        self.command_gateway("/home/Backups/gera_backups_arquivos.sh")
        self.sftp_client.chdir('/home/Backups/backup_{}/backup_{}-{}/'.format(self.date.strftime('%Y'), self.date.strftime("%m"), self.date.strftime('%Y')))
        self.sftp_client.get('backup_{}-{}-{}.tar.gz'.format(self.date.strftime('%d'), self.date.strftime('%m'), self.date.strftime('%Y')),
                             r'C:\Backup\{}_backup_{}-{}-{}.tar.gz'.format(name_gateway, self.date.strftime('%d'), self.date.strftime('%m'), self.date.strftime('%Y')))
        self.sftp_client.close()
        self.ssh.close()
        return True

    def cat_gateway(self, cat_file, grep=None):
        results = {'cat' : self.command_gateway("cat /{} | grep '{}'".format(cat_file, grep) if grep != None else "cat /{}".format(cat_file))}
        return self.format_result(results, remove={})
    
    def chmod_gateway(self, path_file, permission):
        self.command_gateway('chmod {} {}'.format(permission, path_file))
        return 'Successfully chmod !'
    
    def copy_gateway(self, file, path):
        self.command_gateway('cp -r -p {} {}'.format(file, path_move))
        return 'Successfully Copy !'
    
    def create_file_gateway(self, remote_path, content):
        sftp = self.connection_sftp()
        with sftp.open(remote_path, "w") as file:
            file.write(content)
        command = f"vi {remote_path} -c 'wq'"
        self.command_gateway(command)
        return 'File saved !'
    
    def create_folder(self, remote_path):
        self.command_gateway('mkdir {}'.format(remote_path))
        return 'Folder created successfully !'
        
    def csv_gateway(self, path, name_cols_csv, name_cols_del=None, tail=None, grep=None):
        csv = open("arquivo.csv", 'wb')
        tail_value = {
                        (None, None): (1000, None),
                        (None, grep): (1000, grep),
                        (tail, None): (tail, None),
                        (tail, grep): (tail, grep),
                     }
        csv.write(self.tail_gateway(path, *tail_value[(tail, grep)])['tail'])
        csv.close()
        return self.manipulation_csv_to_json('arquivo.csv', name_cols_csv, name_cols_del)
        
    def firewall_gateway(self, communication_ip="'#!/bin/bash' -v"):
        results = {'firewall' : self.command_gateway("cat /home/firewall | grep {}".format(communication_ip))}
        return self.format_result(results, remove={})
    
    def ifconfig_gateway(self):
        results = {'ifconfig' : self.command_gateway('ifconfig | grep inet').strip()}
        return self.format_result(results, remove={'    ':''})
    
    def information_gateway(self):
        INFO_COMMAND = {
                            'name_gateway': 'hostname',
                            'model_cpu' : "lscpu | grep 'Model name'",
                            'cpu' : "lscpu | grep 'CPU(s)'  | grep -v 'On-line' | grep -v 'NUMA'",
                            'uptime': 'uptime  | cut -c 2-27',
                            'memory': "free | grep 'Mem' | cut -c 6-39",
                            'hd': "df -h | awk '{}'".format('{if($(NF) == "/") {print $(NF-1); exit;}}'),
                        }
        results = {}
        for command in INFO_COMMAND:
            results[command] = self.command_gateway(INFO_COMMAND[command])
        return self.format_result(results, remove={'      ':''})

    def ls_gateway(self, path, grep=None):
        results = {'ls' : self.command_gateway("ls /{} | grep '{}'".format(path, grep) if grep != None else "ls /{}".format(path))}
        return self.format_result(results, remove={})
    
    def move_gateway(self, file, path_move):
        self.command_gateway('mv {} {}'.format(file, path_move))
        return 'Successfully Moved !'
    
    def ping_gateway(self, ip_destination, packages):
        results = {'test_ping' : self.command_gateway('ping -c {} {}'.format(packages, ip_destination))}
        return self.format_result(results, remove={'     ':''})

    def rm_gateway(self, file):
        self.command_gateway('rm -rf {}'.format(file))
        return 'Successfully Delete !'
    
    def tail_gateway(self, path, tail, grep=None):
        results = {'tail' : self.command_gateway('tail -{} /{} | grep {}'.format(tail, path, grep)) if grep != None else self.command_gateway('tail -{} /{}'.format(tail, path))}
        return results
    
    def tcpdump_gateway(self, file_name, ip_dump, port_dump, whait):
        self.connection_linux()
        channel_one = self.ssh.invoke_shell()
        channel_one.send('tcpdump host {} and port {} -w {}.pcap'.format(ip_dump, port_dump, file_name) + '\n')
        time.sleep(whait)
        time.sleep(5)
        channel_one.close()
        self.connection_sftp()
        self.command_gateway('tar -czvf {}.tar.gz {}.pcap'.format(file_name, file_name))
        file = file_name.split('/')
        self.sftp_client.get('{}.tar.gz'.format(file_name),
                             r'C:\Backup\Dump\{}.tar.gz'.format(file[2]))
        self.ssh.close()
        return True
    
    def traceroute_gateway(self, ip_destination, packages):
        test_mtr = self.command_gateway('mtr -rw -c {} {} > /home/mtr-report'.format(packages, ip_destination))
        results = {'test_mtr' : self.command_gateway('cat /home/mtr-report')}
        return self.format_result(results, remove={})
    
    def scp_between_gateway(self, file, ip_destination, send_to):
        return self.command_gateway('scp -P5022 {} root@{}:{}'.format(file, ip_destination, send_to))


#SCP -- Criado - Necessário teste -- Vai da erro pois precisa inserir senha


class Manager_Asterisk(Manager_Linux):


    OCCUPATION_COMMANDS = {
        'total_traffic': "asterisk -rx 'core show channels count' | grep 'active call' | awk '{print $1}'",
        'answered': "asterisk -rx 'core show channels' | grep AppDial | grep Up | wc -l",
        'providers_occupation': "asterisk -rx 'core show channels' | grep Dial | cut -c 5-20 | cut -d'-' -f1 | sort | uniq -c",
        'providers_answered': "asterisk -rx 'core show channels' | grep Up | cut -c 5-20 | cut -d'-' -f1 | sort | uniq -c",
    }

    PEERS_COMMANDS = {
        'providers_peers': "asterisk -rx 'sip show peers' | grep -v 'Unmonitored' | grep -v 'Name/username'",
    }

    def __init__(self, gateway, port, username, password):
        super().__init__(gateway, port, username, password)
        
    def log_gateway(self, grep):
        command = 'asterisk -r | grep {}'.format(grep)
        self.connection_linux()
        channel = self.ssh.invoke_shell()
        channel.send(command + '\n')
        output = ''
        time.sleep(60)
        while not channel.closed:
            if channel.recv_ready():
                output += channel.recv(10240).decode('utf-8')
                if ']#' in output:
                    break
        channel.close()
        self.ssh.close()

        with open('output.txt', 'w') as file:
            file.write(output)

        return output
    
    def peers_gateway(self, remove={'     ':'', 'Yes':'', '5060':'', 'OK':'', ')':'', '(':''}):
        results = {}
        for command in self.PEERS_COMMANDS:
            results[command] = self.command_gateway(self.PEERS_COMMANDS[command])
            if results[command] == TimeoutError: return 'NO CONNECTION'
        results = self.format_result(results, remove)
        providers_peers_list = [list_clear.split() for list_clear in results['providers_peers']]
        
        providers_dict = {}
        for provider in providers_peers_list:
            providers_dict[provider[0]] = {'provider_ip': provider[1], 
                                           'provider_status': provider[2] if len(provider) == 3 else provider[2] + provider[3]}
        return providers_dict
        
    def occupation_gateway(self, remove={'     ':''}):
        results = {}
        for command in self.OCCUPATION_COMMANDS:
            results[command] = self.command_gateway(self.OCCUPATION_COMMANDS[command])
            if results[command] == TimeoutError: return 'NO CONNECTION'

        results = self.format_result(results, remove)
        results['total_traffic'], results['answered'] = int(results['total_traffic'][0]), int(results['answered'][0])
        providers_occupation_list = [list_clear.split(' ')[::-1] for list_clear in results['providers_occupation']]
        providers_answered_list = [list_clear.split(' ')[::-1] for list_clear in results['providers_answered']]

        providers_dict = {}
        for provider in providers_occupation_list:
            providers_dict[provider[0]] = {'occupation': int(provider[1]), 'answered': 0}

        for answered in providers_answered_list:
            if answered[0] in providers_dict:
                providers_dict[answered[0]]['answered'] = int(answered[1])
            else:
                providers_dict[answered[0]] = {'occupation':0, 'answered':int(answered[1])}

        results['providers_data'] = providers_dict
        del results['providers_occupation']
        del results['providers_answered']

        return results


    #### TESTE Manager_Linux ####

#test = Manager_Linux('IP_Gateway', port_connection, 'user_acess', 'password_acess')   # TESTE

### Backup Gateway ('Nome do Gateway')
#print(test.backup_gateway('MG_189'))

### Informaçãoes sobre os gateways
#print(test.information_gateway())

### Teste de ping ('IP Ping', 'Números de pacotes')
#print(test.ping_gateway('186.248.146.105', 5))

### Teste MTR ('IP MTR', 'Números de pacotes')
#print(test.traceroute_gateway('8.8.8.8', 5))

### Verificação IPs Firewall ('IP Busca' != Exibe arquivo)
#print(test.firewall_gateway('8.8.8.8'))

### Cat Gateway ('Caminho do arquivo', 'Filtro Cat')
#print(test.cat_gateway('etc/asterisk/extensions.conf'))

### Ls com filtro ('caminho', 'filtro' != ls geral)
#print(test.ls_gateway('home/', 'al')) 

### Tail gateway ('total de linhas', 'caminho do arquivo', 'filtro Tail')
#print(test.tail_gateway(1000, 'var/log/asterisk/cdr-csv/Master.csv')['tail'])

### Configurações de rede gateway
#print(test.ifconfig_gateway())

### CSV - retorno json ('Caminho arquivo', 'Nome das colunas', remover cols nomes atribuidos, 'Qtd Tail', 'Filtro grep')
#cdr_cols_csv = ["NaN", "From-CLI", "Dial", "Contexto", "From-Interno", "Peer-In", "Peer-Out", "Status/Hangup", "Rota / Release", "Inicial", "Atendimento", "Encerramento", "Channel-Duration", "Duração Atendindo" , "Status", "Doc", "Number", "NaN2"]
#cdr_cols_del = ['NaN2', 'Number', 'Doc', 'NaN', 'Contexto'] 
#print(test.csv_gateway('var/log/asterisk/cdr-csv/Master.csv', cdr_cols_csv, cdr_cols_del, 1000)) 

### Move ('Local e nome do arquivo mover', 'Local direcionar arquivo')
#test.move_gateway('/home/tsbc_teste3.pcap', '/home/pcap/')

### Copy  ('Local e nome do arquivo copiar', 'Local direcionar arquivo')
#test.copy_gateway('/home/tsbc_teste3.pcap', '/home/pcap/')

### TCP Dump ('Local para salvar e nome do arquivo', 'IP Dump', 'Port Dump', 'Tempo dump')
#print(test.tcpdump_gateway('/home/test_remote', '216.53.4.1', 5060, 60))

### Vi - Criação de arquivos ('local e nome do arquivo, conteudo do arquivo')
#content = """ Teste de arquivo escrito remotamente!"""
#print(test.create_file_gateway('/home/file_remoto', content))

## Create Folder ('Caminho e nome da pasta')
#print(test.create_folder('/home/pasta_remoto'))

In [ ]:
class Manager_Asterisk(Manager_Linux):


    OCCUPATION_COMMANDS = {
        'total_traffic': "asterisk -rx 'core show channels count' | grep 'active call' | awk '{print $1}'",
        'answered': "asterisk -rx 'core show channels' | grep AppDial | grep Up | wc -l",
        'providers_occupation': "asterisk -rx 'core show channels' | grep Dial | cut -c 5-20 | cut -d'-' -f1 | sort | uniq -c",
        'providers_answered': "asterisk -rx 'core show channels' | grep Up | cut -c 5-20 | cut -d'-' -f1 | sort | uniq -c",
    }

    PEERS_COMMANDS = {
        'providers_peers': "asterisk -rx 'sip show peers' | grep -v 'Unmonitored' | grep -v 'Name/username'",
    }

    def __init__(self, gateway, port, username, password):
        super().__init__(gateway, port, username, password)
        
    def log_gateway(self, grep):
        command = 'asterisk -r | grep {}'.format(grep)
        self.connection_linux()
        channel = self.ssh.invoke_shell()
        channel.send('clear' + '\n')
        channel.send(command + '\n')
        output = ''
        time.sleep(60)
        while not channel.closed:
            if channel.recv_ready():
                output += channel.recv(10240).decode('utf-8')
                if ']#' in output:
                    break
        channel.close()
        self.ssh.close()

        with open('output.txt', 'w') as file:
            file.write(output)

        return output
    
    def peers_gateway(self, remove={'     ':'', 'Yes':'', '5060':'', 'OK':'', ')':'', '(':''}):
        results = {}
        for command in self.PEERS_COMMANDS:
            results[command] = self.command_gateway(self.PEERS_COMMANDS[command])
            if results[command] == TimeoutError: return 'NO CONNECTION'
        results = self.format_result(results, remove)
        providers_peers_list = [list_clear.split() for list_clear in results['providers_peers']]
        
        providers_dict = {}
        for provider in providers_peers_list:
            providers_dict[provider[0]] = {'provider_ip': provider[1], 
                                           'provider_status': provider[2] if len(provider) == 3 else provider[2] + provider[3]}
        return providers_dict
        
    def occupation_gateway(self, remove={'     ':''}):
        results = {}
        for command in self.OCCUPATION_COMMANDS:
            results[command] = self.command_gateway(self.OCCUPATION_COMMANDS[command])
            if results[command] == TimeoutError: return 'NO CONNECTION'

        results = self.format_result(results, remove)
        results['total_traffic'], results['answered'] = int(results['total_traffic'][0]), int(results['answered'][0])
        providers_occupation_list = [list_clear.split(' ')[::-1] for list_clear in results['providers_occupation']]
        providers_answered_list = [list_clear.split(' ')[::-1] for list_clear in results['providers_answered']]

        providers_dict = {}
        for provider in providers_occupation_list:
            providers_dict[provider[0]] = {'occupation': int(provider[1]), 'answered': 0}

        for answered in providers_answered_list:
            if answered[0] in providers_dict:
                providers_dict[answered[0]]['answered'] = int(answered[1])
            else:
                providers_dict[answered[0]] = {'occupation':0, 'answered':int(answered[1])}

        results['providers_data'] = providers_dict
        del results['providers_occupation']
        del results['providers_answered']

        return results

    
    
    #### TESTE Manager_Asterisk ####

#test = Manager_Asterisk('IP_Gateway', port_connection, 'user_acess', 'password_acess') # TESTE


#print(test.occupation_gateway()['providers_occupation'])
#print(test.occupation_gateway())
#print(test.peers_gateway())
#print(test.log_gateway(filtro_log))
